In [1]:
#"/vol3/mag/Unpaywall/Fulltext/pdfs/computerscience/grobid-output"

In [2]:
import numpy as np
import pandas as pd
import os

In [3]:
#os.chdir("/vol3/kd-seminar-ie/")
sparql_df = pd.read_csv("sparql2.csv", sep = ',')

In [4]:
sparql_df['doc_id'] = sparql_df['paper'].str.split('/', n = 5, expand = True).iloc[:, 4]
sparql_df['year'] = sparql_df['paperPubDate'].str.split('-', n = 3, expand = True).iloc[:, 0]
sparql_df = sparql_df.drop(["paper", "paperPubDate"], axis = 1)
sparql_df.head()

,doc_id,year
0,4528552,2009
1,4529005,2011
2,4538592,2007
3,4554164,2014
4,4563250,2014


In [5]:
len(sparql_df)

807709

In [6]:
import glob, os
os.chdir("/vol3/mag/Unpaywall/Fulltext/pdfs/computerscience/grobid-output")
txtfiles = []
for file in glob.glob("*.tei.xml"):
    txtfiles.append(file)
os.chdir("/vol3/kd-seminar-ie/")

In [7]:
filenames_df = pd.DataFrame(txtfiles,columns=['filename'])
filenames_df.head()

,filename
0,1978552532.tei.xml
1,2010754144.tei.xml
2,2581015818.tei.xml
3,2804009286.tei.xml
4,2006990202.tei.xml


In [8]:
filenames_df = filenames_df['filename'].str.split('.', n = 1, expand = True)
filenames_df.columns = ['filename', 'fileformat']
filenames_df = filenames_df.drop("fileformat", axis = 1)
filenames_df.head()

,filename
0,1978552532
1,2010754144
2,2581015818
3,2804009286
4,2006990202


In [23]:
# compare both frames by doc id
joined_df = pd.merge(sparql_df, filenames_df, left_on='doc_id', right_on='filename', how='left').dropna()
joined_df = joined_df[joined_df.year != '2019']
joined_df['filename'] = joined_df['doc_id'] + '.tei.xml'
joined_df.head()

,doc_id,year,filename
22,4703903,2014,4703903.tei.xml
56,4954057,2011,4954057.tei.xml
142,23717612,2013,23717612.tei.xml
169,23965926,2011,23965926.tei.xml
192,24141780,2006,24141780.tei.xml


In [24]:
sample = joined_df.groupby('year', group_keys=False).apply(lambda x: x.sample(min(len(x), 1790), random_state = 42))
print(len(sample))
sample.head()

25060


,doc_id,year,filename
241792,2118681326,2005,2118681326.tei.xml
360321,2161668503,2005,2161668503.tei.xml
124436,1544319464,2005,1544319464.tei.xml
110199,1535853290,2005,1535853290.tei.xml
335761,2107544712,2005,2107544712.tei.xml


In [25]:
# check if each year has same number of documents
sample.groupby('year', group_keys=False).agg(['count'])

,doc_id,filename
,count,count
year,,
2005,1790,1790
2006,1790,1790
2007,1790,1790
2008,1790,1790
2009,1790,1790
2010,1790,1790
2011,1790,1790
2012,1790,1790


In [26]:
sample1 = sample[sample['year'] <= '2010']
sample2 = sample[sample['year'] > '2010']

In [45]:
print(len(sample1))
print(len(sample2))

10740
14320


In [30]:
sample1.head()

,doc_id,year,filename
241792,2118681326,2005,2118681326.tei.xml
360321,2161668503,2005,2161668503.tei.xml
124436,1544319464,2005,1544319464.tei.xml
110199,1535853290,2005,1535853290.tei.xml
335761,2107544712,2005,2107544712.tei.xml


In [38]:
from shutil import copyfile, copy

In [43]:
copy("/vol3/mag/Unpaywall/Fulltext/pdfs/computerscience/grobid-output/" + sample1['filename'].iloc[0],
         "/vol3/kd-seminar-ie/mag_papers_sample1/")

'/vol3/kd-seminar-ie/mag_papers_sample1/2118681326.tei.xml'

In [44]:
for index, row in sample1.iterrows():
    source = "/vol3/mag/Unpaywall/Fulltext/pdfs/computerscience/grobid-output/" + row['filename']
    destination = "/vol3/kd-seminar-ie/mag_papers_sample1/"
    copy(source, destination)

In [46]:
for index, row in sample2.iterrows():
    source = "/vol3/mag/Unpaywall/Fulltext/pdfs/computerscience/grobid-output/" + row['filename']
    destination = "/vol3/kd-seminar-ie/mag_papers_sample2/"
    copy(source, destination)

In [47]:
# check if current directory is save to write:
os.getcwd()

'/vol3/kd-seminar-ie'

In [48]:
sample1['doc_id'].to_csv("document_id_sample1_per_year.csv", index = False)
sample2['doc_id'].to_csv("document_id_sample2_per_year.csv", index = False)

In [19]:
# create list of file names to use for classification task
#sample.drop('filename', axis = 1)
sample['doc_id'].to_csv("document_id_sampled_per_year.csv", index = False)